"""
Create a complete ATM simulator with the following features:

1. Account Management:
   - Create account with PIN
   - Login with account number and PIN
   - Multiple accounts support

2. Banking Operations:
   - Deposit money
   - Withdraw money (with overdraft protection)
   - Check balance
   - Transfer between accounts
   - Transaction history

3. File Storage:
   - Save account data to file
   - Load accounts on startup
   - Transaction log file

4. Security Features:
   - PIN validation
   - Maximum 3 login attempts
   - Account lockout after failed attempts

5. Error Handling:
   - Invalid inputs
   - Insufficient funds
   - File errors

Requirements:
- Use dictionaries for account storage
- Use functions for each operation
- Implement proper error handling
- Save data persistently in files
- Create a user-friendly menu system
"""

In [5]:
import time
import json
import os
from datetime import datetime

In [6]:

class ATM:

    ADMIN_PIN = "9999"

    def __init__(self):
        self.accounts = {}
        self.current_account = None
        self.load_accounts()
        self.name = None
        self.pin = None
        self.balance = 0

    # ---------------- Utilities ---------------- #
    def set_name(self):
        while True:
            name = input("Enter full name: ").strip()
            if any(char.isdigit() for char in name):
                print("Invalid name: numbers are not allowed.")
                continue
            if len(name.split()) < 2:
                print("Invalid name: must contain at least two words.")
                continue
            return name

    def set_pin(self):
        while True:
            pin = input("Set a 4-digit PIN: ").strip()
            if not pin.isdigit():
                print("PIN must contain only digits.")
                continue
            if len(pin) != 4:
                print("PIN must be exactly 4 digits.")
                continue
            return pin

    def generate_account_number(self):
        if not self.accounts:
            return 1000
        return max(self.accounts.keys()) + 1

    # ---------------- Account Management ---------------- #
    def create_account(self):
        print("\n=== Create New Account ===")
        self.name = self.set_name()
        self.pin = self.set_pin()
        self.current_account = self.generate_account_number()
        self.balance = 0

        self.accounts[self.current_account] = {
            "name": self.name,
            "pin": self.pin,
            "balance": 0,
            "transaction_history": [],
            "locked": False,
            "failed_attempts": 0
        }

        self.save_accounts()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] Account created successfully. Account Number: {self.current_account}")

    def login(self):
        acc_num = input("Enter Account Number: ").strip()
        if not acc_num.isdigit():
            print("Invalid account number.")
            return False
        acc_num = int(acc_num)
        if acc_num not in self.accounts:
            print("Account number not found.")
            return False

        account = self.accounts[acc_num]

        if account.get("locked", False):
            print("This account is locked due to multiple failed login attempts.")
            return False

        for attempt in range(3):
            pin_val = input("Enter PIN: ").strip()
            if pin_val == account["pin"]:
                self.current_account = acc_num
                self.balance = account["balance"]
                account["failed_attempts"] = 0
                self.save_accounts()
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"[{timestamp}] Login successful. Welcome {account['name']}.")
                return True
            else:
                account["failed_attempts"] = account.get("failed_attempts", 0) + 1
                remaining = max(0, 2 - attempt)
                print(f"Incorrect PIN. Attempts left: {remaining}")

                if account["failed_attempts"] >= 3:
                    account["locked"] = True
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    print(f"[{timestamp}] Account locked due to 3 failed attempts.")
                self.save_accounts()
        return False

    # ---------------- Banking Operations ---------------- #
    def deposit(self):
        amount = input("Enter deposit amount: ").strip()
        if not amount.isdigit() or int(amount) <= 0:
            print("Invalid deposit amount.")
            return
        amount = int(amount)
        self.balance += amount
        timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
        self.accounts[self.current_account]["balance"] = self.balance
        self.accounts[self.current_account]["transaction_history"].append(
            f"{timestamp} | Deposit: +{amount}"
        )
        self.save_accounts()
        print(f"{timestamp} | Deposit successful. Your balance: {self.balance}")

    def check_balance(self, amount):
        if amount > self.balance:
            print("Insufficient balance.")
            return False
        return True

    def withdraw(self):
        amount = input("Enter withdrawal amount: ").strip()
        if not amount.isdigit() or int(amount) <= 0:
            print("Invalid withdrawal amount.")
            return
        amount = int(amount)
        if not self.check_balance(amount):
            return

        self._simulate_processing("Withdrawal")
        self.balance -= amount
        timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
        self.accounts[self.current_account]["balance"] = self.balance
        self.accounts[self.current_account]["transaction_history"].append(
            f"{timestamp} | Withdraw: -{amount}"
        )
        self.save_accounts()
        print(f"{timestamp} | Withdrawal completed. Your balance: {self.balance}")

    def transfer(self):
        recipient = input("Enter recipient account number: ").strip()
        amount = input("Enter transfer amount: ").strip()

        if not recipient.isdigit() or not amount.isdigit():
            print("Invalid input.")
            return

        recipient = int(recipient)
        amount = int(amount)

        if recipient not in self.accounts:
            print("Recipient account not found.")
            return
        if recipient == self.current_account:
            print("Cannot transfer to the same account.")
            return
        if not self.check_balance(amount):
            return

        self._simulate_processing("Transfer")
        self.balance -= amount
        timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
        self.accounts[self.current_account]["balance"] = self.balance
        self.accounts[self.current_account]["transaction_history"].append(
            f"{timestamp} | Transfer to {recipient}: -{amount}"
        )

        self.accounts[recipient]["balance"] += amount
        self.accounts[recipient]["transaction_history"].append(
            f"{timestamp} | Transfer from {self.current_account}: +{amount}"
        )

        self.save_accounts()
        print(f"{timestamp} | Transfer of {amount} to account {recipient} completed. Your balance: {self.balance}")

    def transaction_history(self):
        history = self.accounts[self.current_account]["transaction_history"]
        if not history:
            print("No transactions found.")
            return
        print("\n=== Transaction History ===")
        for i, tx in enumerate(history, 1):
            print(f"{i}. {tx}")

    # ---------------- Admin Mode ---------------- #
    def admin_mode(self):
        pin = input("Enter admin PIN: ").strip()
        if pin != self.ADMIN_PIN:
            print("Access denied.")
            return

        while True:
            print("\n=== ADMIN MENU ===")
            print("1. View all accounts")
            print("2. Unlock account")
            print("3. Delete account")
            print("4. Total bank balance")
            print("5. Exit admin mode")

            choice = input("Choose: ").strip()

            if choice == "1":
                print("\n=== All Accounts ===")
                for acc, data in self.accounts.items():
                    print(f"Account {acc} | Name: {data['name']} | Balance: {data['balance']} | Locked: {data.get('locked', False)}")

            elif choice == "2":
                acc = input("Enter account number to unlock: ").strip()
                if acc.isdigit():
                    acc = int(acc)
                    if acc in self.accounts:
                        self.accounts[acc]["locked"] = False
                        self.accounts[acc]["failed_attempts"] = 0
                        self.save_accounts()
                        print(f"Account {acc} unlocked successfully.")
                    else:
                        print("Account not found.")
                else:
                    print("Invalid account number.")

            elif choice == "3":
                acc = input("Enter account number to delete: ").strip()
                if acc.isdigit():
                    acc = int(acc)
                    if acc in self.accounts:
                        del self.accounts[acc]
                        self.save_accounts()
                        print(f"Account {acc} deleted successfully.")
                    else:
                        print("Account not found.")
                else:
                    print("Invalid account number.")

            elif choice == "4":
                total = sum(acc["balance"] for acc in self.accounts.values())
                print(f"Total bank balance across all accounts: {total}")

            elif choice == "5":
                print("Exiting admin mode...")
                break
            else:
                print("Invalid choice. Please select a valid option.")

    # ---------------- File Handling ---------------- #
    def save_accounts(self):
        with open("accounts.json", "w") as file:
            json.dump(self.accounts, file, indent=4)

    def load_accounts(self):
        if not os.path.exists("accounts.json"):
            with open("accounts.json", "w") as f:
                json.dump({}, f)
            self.accounts = {}
            return

        with open("accounts.json", "r") as file:
            data = json.load(file)

        self.accounts = {}
        for k, v in data.items():
            acc_num = int(k)
            v.setdefault("locked", False)
            v.setdefault("failed_attempts", 0)
            v.setdefault("transaction_history", [])
            self.accounts[acc_num] = v

    # ---------------- Processing Simulation ---------------- #
    def _simulate_processing(self, operation_name):
        """Simulate a processing animation for transactions."""
        print(f"{operation_name} in progress", end="", flush=True)
        for _ in range(3):
            time.sleep(0.5)
            print(".", end="", flush=True)
        print(" Done.")

    # ---------------- Main Menu ---------------- #
    def run(self):
        print("=== WELCOME TO EASYATM SYSTEM ===")

        while True:
            if self.current_account is None:
                print("\n1. Login")
                print("2. Create Account")
                print("3. Admin Mode")
                print("4. Exit")

                choice = input("Choose: ").strip()

                if choice == "1":
                    self.login()
                elif choice == "2":
                    self.create_account()
                elif choice == "3":
                    self.admin_mode()
                elif choice == "4":
                    print("Thank you for using EasyATM. Have a nice day!")
                    break
                else:
                    print("Invalid choice. Please select a valid option.")
            else:
                print(f"\nLogged in as Account {self.current_account} - {self.accounts[self.current_account]['name']}")
                print("1. Deposit")
                print("2. Withdraw")
                print("3. Check Balance")
                print("4. Transfer")
                print("5. Transaction History")
                print("6. Logout")

                choice = input("Choose: ").strip()

                if choice == "1":
                    self.deposit()
                elif choice == "2":
                    self.withdraw()
                elif choice == "3":
                    print(f"Current balance: {self.balance}")
                elif choice == "4":
                    self.transfer()
                elif choice == "5":
                    self.transaction_history()
                elif choice == "6":
                    print(f"Logging out from account {self.current_account}...")
                    self.current_account = None
                else:
                    print("Invalid choice. Please select a valid option.")




In [7]:
if __name__ == "__main__":
    atm = ATM()
    atm.run()

=== WELCOME TO EASYATM SYSTEM ===

1. Login
2. Create Account
3. Admin Mode
4. Exit


Choose:  1
Enter Account Number:  1000


Account number not found.

1. Login
2. Create Account
3. Admin Mode
4. Exit


Choose:  2



=== Create New Account ===


...

[2026-01-15 10:45:41] Login successful. Welcome Eman Mansour.

Logged in as Account 1000 - Eman Mansour
1. Deposit
2. Withdraw
3. Check Balance
4. Transfer
5. Transaction History
6. Logout


Choose:  6


Logging out from account 1000...

1. Login
2. Create Account
3. Admin Mode
4. Exit


Choose:  4


Thank you for using EasyATM. Have a nice day!


In [8]:
"""
import os

file_path = "accounts.json"

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted successfully.")
else:
    print(f"{file_path} does not exist.")
"""

'\nimport os\n\nfile_path = "accounts.json"\n\nif os.path.exists(file_path):\n    os.remove(file_path)\n    print(f"{file_path} has been deleted successfully.")\nelse:\n    print(f"{file_path} does not exist.")\n'